<a href="https://colab.research.google.com/github/sandyboss0/Indox-acheiver/blob/main/twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [ ]:
!mkdir =p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘=p’: File exists
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 96% 78.0M/80.9M [00:03<00:00, 29.5MB/s]
100% 80.9M/80.9M [00:03<00:00, 24.1MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


Importing the dependencies


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Processing

In [ ]:
twitter_dataset = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [ ]:
twitter_dataset.shape

(1599999, 6)

In [ ]:
twitter_dataset.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
column_names = ['target' , 'id' ,'date' ,'flag' ,'user' ,'text']
twitter_dataset = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',names=column_names)

In [ ]:
twitter_dataset.shape

(1600000, 6)

In [ ]:
twitter_dataset.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
twitter_dataset.isnull().sum()

,0
target,0
id,0
date,0
flag,0
user,0
text,0


In [ ]:
twitter_dataset['target'].value_counts()

,count
target,
0,800000
4,800000


convert the target 4 to 1

In [ ]:
twitter_dataset['target'].replace(4,1,inplace=True)

<ipython-input-24-38aa31e4722c>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  twitter_dataset['target'].replace(4,1,inplace=True)


In [ ]:
twitter_dataset['target'].value_counts()

,count
target,
0,800000
1,800000


In [ ]:
1
port_stem = PorterStemmer()

In [ ]:
def stemming(content):

    stemming_content = re.sub('[a-zA-Z]',' ', content)
    stemming_content = stemming_content.lower()
    stemming_content = stemming_content.split()
    stemmer = PorterStemmer()
    stemming_content = [stemmer.stem(word) for word in stemming_content if not word in set(stopwords.words('english'))]
    stemming_content = ' '.join(stemming_content)

    return stemming_content

In [ ]:
twitter_dataset[stemmed_content] = twitter_dataset['text'].apply(stemming)

NameError: name 'stemmed_content' is not defined

In [ ]:
twitter_dataset['stemmed_content'] = twitter_dataset['text'].apply(stemming)

In [32]:
twitter_dataset.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","@ :// . /2 1 - , ' . . ;"
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,' ... . !
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,@ . 50%
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","@ , ' . ' . ? ' ."


In [33]:
print(twitter_dataset['stemmed_content'])

0          @ :// . /2 1 - , ' . . ;
1                         ' ... . !
2                           @ . 50%
3                                  
4                 @ , ' . ' . ? ' .
                     ...           
1599995                           .
1599996         . - ! â« :// . /~8
1599997                           ?
1599998                      38 !!!
1599999                   # @ @ @ 4
Name: stemmed_content, Length: 1600000, dtype: object


In [34]:
print(twitter_dataset['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [35]:
X = twitter_dataset['stemmed_content'].values
Y = twitter_dataset['target'].values

In [37]:
print(X)

["@ :// . /2 1 - , ' . . ;" "' ... . !" '@ . 50%' ... '?' '38 !!!'
 '# @ @ @ 4']


In [38]:
print(Y)

[0 0 0 ... 1 1 1]


In [39]:
X_train ,X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [40]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [41]:
print(X_test)

["@ - ! ' . & ; ." '/ & ;' '@ !' ... '@ 41 ! 4700 !' '' '@ !!!! !']


In [42]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [43]:
print(X_train)

  (5, 3469)	1.0
  (6, 8444)	0.6745653244872989
  (6, 8484)	0.7382151603694854
  (20, 2318)	1.0
  (30, 533)	1.0
  (45, 1447)	1.0
  (46, 740)	1.0
  (61, 739)	1.0
  (74, 5784)	1.0
  (88, 6276)	1.0
  (96, 4499)	0.8008755702940765
  (96, 4235)	0.5988307948879529
  (99, 4055)	1.0
  (106, 7028)	1.0
  (107, 2534)	1.0
  (108, 4318)	1.0
  (109, 1784)	0.75753502473072
  (109, 1)	0.6527945207385148
  (110, 6772)	1.0
  (123, 4773)	0.8330042941047515
  (123, 4825)	0.5532665234794572
  (136, 739)	1.0
  (139, 5644)	1.0
  (142, 6127)	1.0
  (149, 739)	0.45796765131333955
  :	:
  (1279826, 6920)	1.0
  (1279836, 7608)	1.0
  (1279842, 5938)	1.0
  (1279843, 5062)	1.0
  (1279848, 789)	1.0
  (1279849, 3142)	1.0
  (1279851, 3550)	1.0
  (1279861, 1656)	1.0
  (1279863, 4476)	1.0
  (1279864, 2686)	1.0
  (1279880, 2687)	1.0
  (1279894, 1207)	0.5927335051951225
  (1279894, 304)	0.8053986539714999
  (1279900, 4536)	1.0
  (1279913, 5551)	1.0
  (1279914, 3047)	1.0
  (1279917, 739)	1.0
  (1279934, 7455)	0.5615353500148

In [44]:
print(X_test)

  (6, 7029)	1.0
  (15, 2470)	1.0
  (19, 1599)	1.0
  (20, 6667)	1.0
  (23, 3552)	1.0
  (37, 2470)	1.0
  (41, 6968)	1.0
  (54, 902)	1.0
  (61, 6074)	1.0
  (64, 739)	1.0
  (66, 377)	0.5269754385611195
  (66, 2534)	0.45491671156885005
  (66, 3613)	0.7178771988917738
  (78, 1447)	1.0
  (92, 6641)	1.0
  (110, 8270)	1.0
  (113, 974)	1.0
  (114, 4055)	1.0
  (123, 3550)	1.0
  (131, 8270)	1.0
  (149, 4655)	1.0
  (170, 740)	1.0
  (176, 740)	1.0
  (184, 739)	1.0
  (195, 6772)	1.0
  :	:
  (319773, 659)	1.0
  (319822, 7976)	1.0
  (319834, 3550)	1.0
  (319844, 2504)	1.0
  (319849, 6825)	1.0
  (319860, 0)	0.7423996134344486
  (319860, 2504)	0.6699573225007557
  (319862, 6276)	1.0
  (319878, 3550)	0.6308085570638233
  (319878, 4234)	0.7759385055112661
  (319879, 6362)	1.0
  (319887, 739)	1.0
  (319898, 5946)	1.0
  (319907, 3306)	1.0
  (319917, 3469)	1.0
  (319922, 5391)	1.0
  (319929, 825)	1.0
  (319941, 8206)	1.0
  (319952, 1803)	1.0
  (319961, 6565)	1.0
  (319970, 1447)	1.0
  (319982, 3140)	1.0
  (31

In [45]:
model.fit(X_train,Y_train)

NameError: name 'model' is not defined

In [46]:
model.fit(X_train,Y_train)

NameError: name 'model' is not defined

In [47]:
model = LogisticRegression(max_iter=1000)

In [48]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [49]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [50]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.51560859375


In [51]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [52]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.510903125


In [53]:
import pickle

In [56]:
filenmae = 'trained_model_.sav'
pickle.dump(model,open(filenmae,'wb')) # Changed 'filename' to 'filenmae'

In [57]:
loaded_model = pickle.load(open(filenmae,'rb'))

In [58]:
X_new  = X_test[200]
print(Y_test[200])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('Negative')

else:
  print('Positive')

1
[0]
Negative


In [59]:
X_new  = X_test[3]
print(Y_test[3])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('Negative tweet')

else:
  print('Positive tweet')

0
[0]
Negative tweet
